In [1]:
import pandas as pd
import datetime as dt
import urllib.parse as urlparse
from time import sleep

from fyers_api import fyersModel, accessToken
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
client_id = 'J6QOEMU1XS-100'
secret_key = 'LN566GCZLS'
redirect_uri = 'http://127.0.0.1:5000/login'

In [3]:
app_id = 'J6QOEMU1XS-100'
app_secret = 'LN566GCZLS'
redirect_url = 'http://127.0.0.1:5000/login'
user_id = 'XS00222'
password = 'srijita23$'
two_fa = 'APWPB4631H'


def get_token():
    session = accessToken.SessionModel(client_id=app_id,
                                       secret_key=app_secret,
                                       redirect_uri=redirect_url,
                                       response_type="code",
                                       grant_type="authorization_code")
    url = session.generate_authcode()
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(executable_path='./chromedriver', options=options)

    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, '//div[@class="container login-main-start"]')))
    driver.find_element_by_xpath("//input[@id='fyers_id']").send_keys(user_id)
    driver.find_element_by_xpath("//input[@id='password']").send_keys(password)
    driver.find_element_by_xpath("//input[@id='pancard']").send_keys(two_fa)
    driver.find_element_by_xpath("//button[@id='btn_id']").click()
    sleep(2)
    current_url = driver.current_url
    driver.close()

    parsed = urlparse.urlparse(current_url)
    auth_code = urlparse.parse_qs(parsed.query)['auth_code'][0]
    session.set_token(auth_code)
    response = session.generate_token()
    access_token = response['access_token']
    return access_token

In [4]:
access_token = get_token()

In [5]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)

In [18]:
def fetchOHLC(symbol, time):
    data = {"symbol":symbol,"resolution":time,"date_format":"1","range_from":str(dt.datetime.now().date() -dt.timedelta(days=4)),"range_to":str(dt.datetime.now().date() - dt.timedelta(days=2)),"cont_flag":'1'}
    data = fyers.history(data)
    data = pd.DataFrame(data['candles'])
    return data

In [ ]:
def add_indicator(data):
    EMA5  = EMAIndicator(close = Data["close"], window = 5, fillna = True)
    EMA8  = EMAIndicator(close = Data["close"], window = 8, fillna = True)
    SMA20Volume = SMAIndicator(close= Data["volume"], window=20, fillna= True)
    SMA20OI = SMAIndicator(close= Data["oi"], window=20, fillna= True)
    RSI = RSIIndicator(close= Data["close"], window=14, fillna= True)
    VWAP = VolumeWeightedAveragePrice(Data['high'], Data['low'], Data['close'], Data['volume'], 14, True)
    
    
    Data['RSI']= RSI.rsi()
    Data['EMA8']= EMA8.ema_indicator()
    Data['EMA5']= EMA5.ema_indicator()
    Data['SMA20Volume']= SMA20Volume.sma_indicator()
    Data['SMA20OI']=SMA20OI.sma_indicator()
    Data['Vwap']=VWAP.volume_weighted_average_price()
    
    return Data
    

In [ ]:
def day(timestamp):
    timestamp = str(timestamp)
    return timestamp.split(" ")[0]

In [ ]:
def VWAP(data):
    date = str(dt.datetime.now()).split(' ')[0]
    data['day'] = data['date'].apply(day)
    d1 = data.loc[data['day']==date]
    d1['price_volume'] = ((d1['high'] + d1['close'] + d1['low'])/3) * d1['volume']
    vwap = d1['price_volume'].sum()/d1['volume'].sum()
    return vwap

In [ ]:
# If you want to remove condition write true inside -- if(True)
def check_intraday_conditions(Data,j):
    if(Data["RSI"][j] > 50):
        if(Data["close"][j] > VWAP(Data)):
            if(Data['close'][j]>Data['open'][j]):
                if(Data["volume"][j]>=1.5*Data['SMA20Volume'][j]):
                    if(Data['oi'][j]<Data['SMA20OI'][j]):
                        if(Data['EMA5'][j]>Data['EMA8'][j] and (Data['EMA5'][j-4]< Data['EMA8'][j-4] or Data['EMA5'][j-3]< Data['EMA8'][j-3] or Data['EMA5'][j-2]< Data['EMA8'][j-2] or Data['EMA5'][j-1]< Data['EMA8'][j-1])):
                            return True
                        
    return False

In [10]:
bn = fetchOHLC("NSE:NIFTYBANK-INDEX", 5)
nf = fetchOHLC('NSE:NIFTY50-INDEX', 5)

ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2021-10-09.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2021-10-09.txt'


In [19]:
d = fetchOHLC('NSE:SBIN21OCTFUT', 5)
d.head()

ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2021-10-09.txt'


,0,1,2,3,4,5
0,1633405500,461.40,462.25,459.30,461.10,1923000
1,1633405800,461.10,462.00,460.55,462.00,1236000
2,1633406100,462.00,463.60,461.95,462.45,687000
3,1633406400,462.45,462.45,460.85,461.10,472500
4,1633406700,461.30,464.30,461.15,463.75,606000


In [11]:
bn.head()

,0,1,2,3,4,5
0,1633405500,37499.8,37516.50,37364.75,37435.0,0
1,1633405800,37436.5,37536.65,37411.50,37524.8,0
2,1633406100,37525.4,37546.00,37488.45,37519.6,0
3,1633406400,37518.8,37519.65,37430.55,37457.1,0
4,1633406700,37459.1,37527.50,37448.95,37503.4,0


In [ ]:
while(True):
    # Enter the expiry date here in quotes 'Strike Price'
    Expiry_Date='21909'
    # BANKNIFTY CE
    spot= bn['close'][bn.shape[0]-1]
    atm = (math.floor(spot/100))*100
    itm1 = atm-100
    itm2= itm1-100
    itm3= itm2-100
    itm4= itm3-100
    itm5= itm4-100
    strike= [atm,itm1,itm2,itm3,itm4,itm5]
    for i in strike:
        strike = "BANKNIFTY"+ Expiry_Date + str(i) + 'CE'
        Data = fetchOHLC(strike, '1')
        Data=add_indicator(Data)
        if(check_intraday_conditions(Data, Data.shape[0]-1)):
            print("Bank Nifty CE")
            print(str(dt.datetime.now().time()) + "   " + str(i) )
    #BANKNIFTY PE
    otm1=atm+100
    otm2=otm1+100
    otm3=otm2+100
    otm4=otm3+100
    otm5=otm4+100
    strike= [atm,otm1,otm2,otm3,otm4,otm5]
    for i in strike:
        strike = "BANKNIFTY"+ Expiry_Date + str(i) + 'PE'
        Data = fetchOHLC(strike, '1')
        Data = add_indicator(Data)
        if(check_intraday_conditions(Data, Data.shape[0]-1)):
            print("Bank Nifty PE")
            print(str(dt.datetime.now().time()) + "   " + str(i) )
    #NIFTY CE
    spot= nf['close'][nf.shape[0]-1]
    atm = (math.floor(spot/100))*100
    itm1 = atm-100
    itm2= itm1-100
    itm3= itm2-100
    itm4= itm3-100
    itm5= itm4-100
    strike= [atm,itm1,itm2,itm3,itm4,itm5]
    for i in strike:
        strike = "NIFTY"+ Expiry_Date + str(i) + 'CE'
        Data = fetchOHLC(strike, '1')
        Data=add_indicator(Data)
        if(check_intraday_conditions(Data, Data.shape[0]-1)):
            print("Nifty CE")
            print(str(dt.datetime.now().time()) + "   " + str(i) )
    
    #NIFTY PE
    otm1=atm+100
    otm2=otm1+100
    otm3=otm2+100
    otm4=otm3+100
    otm5=otm4+100
    strike= [atm,otm1,otm2,otm3,otm4,otm5]
    for i in strike:
        strike = "NIFTY"+ Expiry_Date + str(i) + 'PE'
        Data = fetchOHLC(strike, '1')
        Data = add_indicator(Data)
        if(check_intraday_conditions(Data, Data.shape[0]-1)):
            print("Nifty PE")
            print(str(dt.datetime.now().time()) + "   " + str(i) )
    time.sleep(10)

In [ ]:
# data = {"symbols":"NSE:SBIN21OCTFUT", 'oiflag':True}
# # fyers.quotes(data['oi'])